In [1]:
import numpy as np
import cv2
from keras.preprocessing import image
import emoji

In [2]:
#-----------------------------
#opencv initialization
face_cascade = cv2.CascadeClassifier('/Users/apple/Downloads/opencv-master/data/haarcascades_cuda/haarcascade_frontalface_default.xml')

In [3]:
#-----------------------------
#face expression recognizer initialization
from keras.models import model_from_json
model = model_from_json(open("/Users/apple/Downloads/tensorflow-101-master/model/facial_expression_model_structure.json", "r").read())
model.load_weights('/Users/apple/Downloads/tensorflow-101-master/model/facial_expression_model_weights.h5') #load weights
#-----------------------------

In [4]:
def draw_border(img, pt1, pt2, color, thickness, r, d):
    x1,y1 = pt1
    x2,y2 = pt2
    # Top left
    cv2.line(img, (x1 + r, y1), (x1 + r + d, y1), color, thickness)
    cv2.line(img, (x1, y1 + r), (x1, y1 + r + d), color, thickness)
    cv2.ellipse(img, (x1 + r, y1 + r), (r, r), 180, 0, 90, color, thickness)
    # Top right
    cv2.line(img, (x2 - r, y1), (x2 - r - d, y1), color, thickness)
    cv2.line(img, (x2, y1 + r), (x2, y1 + r + d), color, thickness)
    cv2.ellipse(img, (x2 - r, y1 + r), (r, r), 270, 0, 90, color, thickness)
    # Bottom left
    cv2.line(img, (x1 + r, y2), (x1 + r + d, y2), color, thickness)
    cv2.line(img, (x1, y2 - r), (x1, y2 - r - d), color, thickness)
    cv2.ellipse(img, (x1 + r, y2 - r), (r, r), 90, 0, 90, color, thickness)
    # Bottom right
    cv2.line(img, (x2 - r, y2), (x2 - r - d, y2), color, thickness)
    cv2.line(img, (x2, y2 - r), (x2, y2 - r - d), color, thickness)
    cv2.ellipse(img, (x2 - r, y2 - r), (r, r), 0, 0, 90, color, thickness)

In [20]:
emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

cap = cv2.VideoCapture(0) #process real time web-cam

angry_emoji = cv2.imread("/Users/apple/Downloads/Face-Emotion-Recognition-master/emojis/angry.png", -1)
disgust_emoji = cv2.imread("/Users/apple/Downloads/Face-Emotion-Recognition-master/emojis/disgust.png", -1)
fear_emoji = cv2.imread("/Users/apple/Downloads/Face-Emotion-Recognition-master/emojis/fearful.png", -1)
happy_emoji = cv2.imread("/Users/apple/Downloads/Face-Emotion-Recognition-master/emojis/happy.png", -1)
neutral_emoji = cv2.imread("/Users/apple/Downloads/Face-Emotion-Recognition-master/emojis/neutral.png", -1)
surprise_emoji = cv2.imread("/Users/apple/Downloads/Face-Emotion-Recognition-master/emojis/surprised.png", -1)
sad_emoji = cv2.imread("/Users/apple/Downloads/Face-Emotion-Recognition-master/emojis/sad.png", -1)

while(True):
    
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors= 5)
  
    for (x,y,w,h) in faces:
        
        if w > 130: #trick: ignore small faces
            
            draw_border(img, (x, y), (x + w, y + h), (255, 0, 105),4, 15, 10) ## draw rectangle around face.
            
            detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
             
            detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
            detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48
     
            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            

            img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]
    
#------------------------------

            predictions = model.predict(img_pixels) #store probabilities of 7 expressions
            max_index = np.argmax(predictions[0])

            emotion = ""
            emotion = "%s" % (emotions[predictions[0].argmax()])
             
            # Display the resulting frame
           
            if emotion == 'neutral':
                img2 = cv2.resize(neutral_emoji, (w, h))
            elif emotion == 'angry':
                img2 = cv2.resize(angry_emoji, (w, h))
            elif emotion == 'disgust':
                img2 = cv2.resize(disgust_emoji, (w, h))
            elif emotion == 'fear':
                img2 = cv2.resize(fear_emoji, (w, h))
            elif emotion == 'happy':
                img2 = cv2.resize(happy_emoji, (w, h))
            elif emotion == 'surprise':
                img2 = cv2.resize(surprise_emoji, (w, h))
            else:
                img2 = cv2.resize(sad_emoji, (w, h))

            alpha = img2[:,:,3]/255.0
            img[y:y+h, x:x+w, 0] = img[y:y+h, x:x+w, 0] * (1-alpha) + alpha * img2[:,:,0]
            img[y:y+h, x:x+w, 1] = img[y:y+h, x:x+w, 1] * (1-alpha) + alpha * img2[:,:,1]
            img[y:y+h, x:x+w, 2] = img[y:y+h, x:x+w, 2] * (1-alpha) + alpha * img2[:,:,2]

                

            cv2.putText(img, emotion, (int(x+w+15), int(y-12+0)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)

#---------------------------------
    cv2.imshow('Window',img)
#---------------------------------

    if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
        break

#kill open cv things
cap.release()
cv2.destroyAllWindows()
